# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/yujie/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/yujie/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/yujie/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.41it/s]
/root/miniconda3/envs/yujie/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中的重要元素。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、强健、刚毅、积极、进取等。

乾卦的卦辞是：“元、亨、利、贞。”这四个字含义非常丰富，元表示开始、创造、第一，亨表示通、顺利，利表示有利、成功，贞表示正、正确。综合起来，乾卦象征着事物发展变化的规律，具有积极向上的动力。

在易经中，乾卦的五行属性为阳、木、木生于火，因此，乾卦也象征着火。同时，乾卦与坤卦形成对比，坤卦象征着地、坤德、承载、顺从等，两卦相互补充，共同构成了天地之象。

在八宫图说中，乾卦位于北方，与事业、努力、刚毅、坚定等有关，同时也与家庭、亲情、领导力等有关。乾卦所代表的能量具有积极向上的动力，对于个人成长和事业发展具有很好的推动作用。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图说、易经、易学中的重要元素。讼卦是由两个阳爻夹一个阴爻构成，象征着争斗、诉讼、诉讼、争端等。

讼卦的卦辞是：“天、 nuclear、无赖。”天表示天地、大自然，核表示核心、中心，無赖表示无赖、流氓。综合起来，讼卦象征着天地之间的争斗，具有强烈的冲突和对抗性。

在易经中，讼卦的五行属性为火、火生于土，因此，讼卦也象征着土。同时，讼卦与乾卦形成对比，乾卦象征着天、干燥、强健、刚毅等，两卦相互补充，共同构成了天地之象。

在八宫图说中，讼卦位于西南，与争斗、诉讼、战争、疾病等有关。讼卦所代表的能量具有强烈的冲突和对抗性，对于个人和团体的影响力非常大。因此，在处理争端和冲突时，需要谨慎对待，避免过于激烈和极端。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250825_201054"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天子的刚强和刚正不阿。在这个卦中，预示着卜者将得到一个吉利的不幸之财，但必须谨慎行事，避免惹起是非。', 'content': '\n乾卦的哲学理念是：天行健，君子拟之。君子需要效法天行的健健之质，行止合一，坚毅不阿，从而获得吉运。乾卦象征天地的力量，强调刚强和刚正不阿， representation of the刚强之质。\n\n在事业方面，乾卦提示君子们必须坚毅果断，行动刚强，坚忍不拔，有所作为。在事业中，适合担任 leadership角色，或者在某个领域具有独特的才能和贡献。在事业中，需要坚定信念，勇敢决策，最新鲜的思维和决策将有助于事业的发展。\n\n在经商方面，乾卦提示经商者需要刚强和不阿，强调诚信和公平，'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_201054）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_201054）：
[gMASK]sop 周易中的讼卦是什么样子? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦是一个古老的占卜卦象，在周易卦象中，它由下卦乾和上卦坎组成。坎象征着北部，乾象征着南部，意味着南部和北部之间的分界线。师卦的核心哲学是“刚柔相济”，它代表着宁静、安详和形势stop（暂时停顿）。', 'content': '\n师卦象征着战争即将开始，但在此之前有一个暂停的时机。它预示着如果能够抓住机遇，便可以获得 Flexibility（灵活性）和 Profit（利润）。师卦适用于各种场合，特别是用于事业、经商和求名。\n\n师卦的核心哲学是“刚柔相济”，代表着一种灵活而强大的结合。因此，在处理事务时，应该灵活机动，抓住机遇，同时保持冷静和理智。在经商时，师卦提示着需要灵活应对市场变化，坚持正道，注重协作与交流。在求名时，师卦强调坚忍不拔，勇往直前，以及不断地学习与提高。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_201054）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3

timestamp = "20250825_204357"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，位于乾卦的卦象中，它是六十四卦中第一卦。乾卦代表天，预示着天地之大顺。根据周易的解释，乾卦预示着大吉，有顺从小吉之象。然而，如果 solo 的话，那么可能不太妙，因为乾卦的阳爻在上，象征君子观者代之，君子之道成于累世。在事业上，乾卦象征大吉，有正派君子之象。在经商方面，icted', 'content': ' 然而，如果 solo 的话，那么可能不太妙，因为乾卦的阳爻在上，象征君子观者代之，君子之道成于累世。在经商方面，经商者应当刚健强健，充满直道，才能取得成功。在外交方面，象征着美好的形势，观者代之可获得大吉。在决策方面，乾卦提醒君子之大道，强调正派、刚健、直ency，要求极高的道德品质和行为。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_204357）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)